## 요약
- 기존 전처리 코드에서는 정수 인코딩한 token들도 scaling을 진행함
- transformer에서 embedding model을 사용하려면 정수 값으로 입력 받아야 함
- 또한 중고 도서 데이터가 아닌 도서 정보만 사용
- 이에 맞게 코드 수정

In [1]:
import os, natsort, re
from tqdm import tqdm
import time, random

In [2]:
from itertools import repeat, chain

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [4]:
#cd /content/drive/MyDrive/AI3_prjct2_aladin/

In [3]:
cd /content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/

[Errno 2] No such file or directory: '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/research


In [3]:
# 로컬에서

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [4]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/'
#PRJCT_PATH = '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
#PRJCT_PATH = '/content/drive/MyDrive/AI3_prjct2_aladin/aladin_usedbook/'
save_dir = 'processed/model_input'
dir_path = os.path.join(PRJCT_PATH,save_dir)
#dir_path = './'

In [5]:
ls

20250211_reg_simulation.ipynb:Zone.Identifier
240710_crawling_step0.ipynb
240711_crawling_step1.ipynb
240711_preprocess_bookinfo.ipynb
240715_encoding_usedinfo.ipynb
240716_check_bookinfo.ipynb
240716_check_bookinfo2.ipynb
240716_encoding_bookinfo.ipynb
240717_simple_model_for_sample.ipynb
240717_split_and_scale.ipynb
240718_step0_by_js.ipynb
240719_additional_eda.ipynb
240719_simple_model_for_cropped.ipynb
240721_GridSearch_for_XGB.ipynb
240721_experiment_w_XGB.ipynb
240721_hyperparameters_XGB.ipynb
241023_basic_model.ipynb
241023_preprocess_for_pred_rglr.ipynb
241024_basic_torch_model.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb:Zone.Identifier
250112_modulized_test_20480_init_lr_analyze2.ipynb
250112_modulized_test_20480_init_lr_analyze2.ipynb:Zone.Identifier
250112_modulized_test_plot_rslt.ipynb
250112_modulized_test_plot_rslt.ipynb:Zone.Identifier
250123_modulized_test_20480_init_lr_test_score.ipynb
250123_modulized_test_20480

In [6]:
import sys
sys.path.append(PRJCT_PATH)

In [7]:
RSLT_DIR = PRJCT_PATH + 'processed/'

bookinfo_name = 'bookinfo_ver{}.pkl'.format(0.75)
bookinfo_path = os.path.join(RSLT_DIR,bookinfo_name)

sys.path.append(PRJCT_PATH)
from module_aladin.file_io import load_pkl, save_pkl
from module_aladin.data_process import pd_datetime_2_datenum

from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.sequence import pad_sequences
import itertools
from sklearn.preprocessing import MinMaxScaler

2025-03-09 20:23:14.716482: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-09 20:23:15.035894: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 20:23:15.051801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-09 20:23:15.051827: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [8]:
def set_corpus_size(freq,size_feat,mode):
    # 입력받은 mode와 size_feat에 따라 size 크기 결정
    if mode == 'uniform':
        cond = freq['counts']>=freq['counts'].iloc[size_feat]
        size = np.sum(cond)
    elif mode =='ths':
        cond = freq[freq['counts'] > size_feat]
        size = np.sum(cond)
    else :
        if size_feat == None : size = len(freq)
#        elif size_feat > len(data) : size = len(freq)
        else : size = size_feat
    return size

def make_encoding_by_freq(freq,null_val='[PAD]',size_feat=None,mode=None):
    #빈도수 기반 정수 인코딩 dict 만들기
    # freq : token 별 등장 빈도 (value_count), size_feat : size관련 변수(max_size, ths등), mode : size 결정 방법
    df_freq = pd.DataFrame(freq).T
    df_freq = df_freq.rename(columns={0:'token',1:'counts'})
    temp = df_freq.sort_values(by='counts',ascending=False)
    size = set_corpus_size(temp,size_feat,mode)
    temp = temp.iloc[:size]
    temp['val'] = np.arange(size)+1
    temp2 = temp.set_index('token').to_dict()
    map_token_encode = temp2['val']
    map_token_encode[null_val]=0
    return map_token_encode

def encode_tokens(map_token,x,oov=True):
    oov_val = len(map_token)+1 if oov else 0 
    return map_token[x] if x in map_token else oov_val

def make_author_encode_map(bookinfo,ths_author):
    pvtb = pd.pivot_table(data=bookinfo,index='Author',values='SalesPoint',aggfunc=np.sum)
    pvtb = pvtb.sort_values(by='SalesPoint',ascending=False)
    author_top_k= pvtb[pvtb['SalesPoint']>=ths_author].index
    encode_author = pd.DataFrame({'author' : author_top_k.values,'val':np.arange(1,len(author_top_k)+1)})
    encode_author = encode_author.set_index('author')
    return encode_author.to_dict()['val']

def make_publshr_encode_map(publshr_data,ths_publshr):
    stats = publshr_data.value_counts().sort_values(ascending=False)
    top_k_val = stats.iloc[ths_publshr]
    publshr_top_k = list(stats[stats >= top_k_val].index)
    return {
        publshr : n+1
        for n,publshr in enumerate(publshr_top_k)
    }
    

def make_store_encode_map(store_data):
    stores= store_data.value_counts().sort_values(ascending=False)
    return {
        place : n+1
        for n,place in enumerate(stores.index)
    }

In [9]:
file_name = 'bookinfo_ver{}.csv'.format(1.0)
file_path = os.path.join(RSLT_DIR,file_name)
bookinfo_raw = pd.read_csv(file_path)

In [10]:
bookinfo_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158084 entries, 0 to 158083
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       158083 non-null  object
 1   ItemId      158084 non-null  int64 
 2   BName_sub   9007 non-null    object
 3   Author      158084 non-null  object
 4   Publshr     158084 non-null  object
 5   Author_mul  158084 non-null  bool  
 6   Pdate       158084 non-null  int64 
 7   RglPrice    158084 non-null  int64 
 8   SlsPrice    158084 non-null  int64 
 9   SalesPoint  158084 non-null  int64 
 10  Category    158084 non-null  object
 11  Source      158084 non-null  object
dtypes: bool(1), int64(5), object(6)
memory usage: 13.4+ MB


In [11]:
cond = bookinfo_raw.RglPrice >= 200000
bookinfo_raw[cond]

,BName,ItemId,BName_sub,Author,Publshr,Author_mul,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Source
4040,도스또예프스끼 전집 - 전25권,233527,NaN,표도르 도스토예프스키,열린책들,True,20000612,360000,324000,92,소설/시/희곡,2000년12월4주
11996,회사연감 2000,241358,NaN,매일경제신문사,매일경제신문사,False,20000630,220000,198000,28,경제경영,2000년8월1주
13163,테마동화 2000 -전42권,262346,NaN,보림 편집부,보림,False,20000101,398000,358200,109,전집/중고전집,2001년10월1주
14481,만화 전략 삼국지 - 전60권,127645,NaN,요코야마 미쓰테루,대현출판사,False,20000101,425000,382500,369,만화,2001년11월2주
20279,토지 - 전21권 세트,324043,NaN,박경리,나남출판,False,20020106,205800,205800,4966,소설/시/희곡,2002년10월2주
21280,"도쿠가와 이에야스 제12,3부 - 전32권 세트",291376,NaN,야마오카 소하치,솔출판사,True,20010810,256000,230400,908,소설/시/희곡,2002년11월5주
26086,프로이트 전집 - 전15권 세트,430579,NaN,지크문트 프로이트,열린책들,True,20030930,225000,202500,621,인문학,2003년10월4주
35000,이이화의 한국사 이야기 1~22 세트,492628,NaN,이이화,한길사,False,20040515,240000,216000,393,역사,2004년5월4주
35475,선생님도 놀란 과학 뒤집기 세트 - 전24권,343940,NaN,성우 편집부,도서출판성우,False,20020401,288000,259200,435,청소년,2004년6월4주
37422,니체 전집 - 전21권,590516,NaN,프리드리히 니체,책세상,True,20051107,470000,423000,512,인문학,2005년10월5주


In [12]:
bookinfo_raw.BName

0               누가 내 치즈를 옮겼을까?
1               부자 아빠 가난한 아빠 1
2                부자 아빠의 투자 가이드
3               부자 아빠 가난한 아빠 2
4                       창가의 토토
                  ...         
158079    백앤아 6 : 저주 걸린 장난감 가게
158080        타키 포오의 이세계 여행사 6
158081                    달샤베트
158082       뉴턴 사과로 우주의 비밀을 열다
158083                 벤처 마인드셋
Name: BName, Length: 158084, dtype: object

In [13]:
'-' in bookinfo_raw.BName.iloc[0]

False

In [14]:
cond = bookinfo_raw.BName.isna()
bookinfo_raw[cond]

,BName,ItemId,BName_sub,Author,Publshr,Author_mul,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Source
155630,NaN,335559202,(협업자들)(고안하는 손)(허용오차),건축사사무소 SoA,건축사사무소 SoA,False,20240228,20000,20000,1329,예술/대중문화,2024년3월3주


In [15]:
cond = bookinfo_raw['BName'].fillna('').apply(lambda x : ' - ' in x)
bookinfo_raw[cond]

,BName,ItemId,BName_sub,Author,Publshr,Author_mul,Pdate,RglPrice,SlsPrice,SalesPoint,Category,Source
30,영어공부 절대로 하지마라! - 일반용 듣기와 받아쓰기,207491,NaN,정찬용,사회평론,True,19991001,9800,8820,5454,외국어,2000년10월1주
32,오래된 정원 - 상,230139,NaN,황석영,창비,False,20000502,15000,13500,5246,소설/시/희곡,2000년10월1주
34,오래된 정원 - 하,230134,NaN,황석영,창비,False,20000502,15000,13500,4968,소설/시/희곡,2000년10월1주
79,영어공부 절대로 하지마라! - 비기너용 듣기와 받아쓰기,225566,NaN,정찬용,사회평론,True,20000301,9800,8820,2842,외국어,2000년10월1주
199,어린이 영어 엄마가 가르쳐라 - 하,98697,NaN,김숙희,조선일보사,False,20040720,13800,12420,644,좋은부모,2000년10월1주
...,...,...,...,...,...,...,...,...,...,...,...,...
157918,2025 Always First 양승아 응급처치학개론 - 전3권,342747405,NaN,양승아,"포러스(구, 비전에듀테인)",False,20240710,49000,44100,3220,수험서/자격증,2024년7월2주
157954,혼공쌤의 초등만화영단어 상.중.하 세트 - 전3권,341965420,NaN,허준석,길벗스쿨,True,20240628,45000,40500,3420,어린이,2024년7월2주
158019,2025 심슨 전략서 + 암기 노트 세트 - 전2권,342880505,NaN,심우철,심슨북스,False,20240715,33000,29700,1210,수험서/자격증,2024년7월2주
158035,장자 - 내편,6735000,NaN,장자,을유문화사,True,20100330,18000,16200,2804,인문학,2024년7월2주


In [16]:
cols = bookinfo_raw.columns.to_list()
x_idxs, y_idx = [0,2,3,4,5,6,9,10], 7
x_cols = [cols[i] for i in x_idxs]
y_col = cols[y_idx]

data_X, data_y  = bookinfo_raw[x_cols], bookinfo_raw[y_col]

In [17]:
from sklearn.model_selection import train_test_split

X_data, X_tst, y_data, y_tst = train_test_split(data_X,data_y,test_size=0.2,random_state=329)
X_trn, X_vld, y_trn, y_vld = train_test_split(X_data,y_data,test_size=0.2,random_state=329)

display(X_trn.columns)
display(X_trn.shape, y_trn.shape)
display(X_vld.shape, y_vld.shape)
display(X_tst.shape, y_tst.shape)

Index(['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate',
       'SalesPoint', 'Category'],
      dtype='object')

(101173, 8)

(101173,)

(25294, 8)

(25294,)

(31617, 8)

(31617,)

In [18]:
data_dict = {
    'trn': {
        'X': X_trn,
        'y': y_trn
        },
    'vld':{
        'X': X_vld,
        'y': y_vld
        },
    'tst':{
        'X': X_tst,
        'y': y_tst
        
    }
}

In [19]:
mecab = Mecab()
tokenizer_basic = lambda x : mecab.morphs(x)
#apply tokenizer
cols_tknz = ['Category','BName','BName_sub']
cols_freq = ['Author','Publshr']

In [20]:
for mode, data in data_dict.items():
    bookinfo = data['X']
    for col in cols_tknz:
        bookinfo[col] = bookinfo[col].fillna('').apply(tokenizer_basic)
    data_dict[mode]['X'] = bookinfo

In [21]:
display(data_dict['trn']['X'].info())
display(data_dict['vld']['X'].info())
display(data_dict['tst']['X'].info())

<class 'pandas.core.frame.DataFrame'>
Index: 101173 entries, 96986 to 69788
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       101173 non-null  object
 1   BName_sub   101173 non-null  object
 2   Author      101173 non-null  object
 3   Publshr     101173 non-null  object
 4   Author_mul  101173 non-null  bool  
 5   Pdate       101173 non-null  int64 
 6   SalesPoint  101173 non-null  int64 
 7   Category    101173 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 6.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 25294 entries, 132734 to 43660
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       25294 non-null  object
 1   BName_sub   25294 non-null  object
 2   Author      25294 non-null  object
 3   Publshr     25294 non-null  object
 4   Author_mul  25294 non-null  bool  
 5   Pdate       25294 non-null  int64 
 6   SalesPoint  25294 non-null  int64 
 7   Category    25294 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 1.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 31617 entries, 146028 to 115151
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       31617 non-null  object
 1   BName_sub   31617 non-null  object
 2   Author      31617 non-null  object
 3   Publshr     31617 non-null  object
 4   Author_mul  31617 non-null  bool  
 5   Pdate       31617 non-null  int64 
 6   SalesPoint  31617 non-null  int64 
 7   Category    31617 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 2.0+ MB


None

In [22]:
#make encoding map
bookinfo = data_dict['trn']['X']
book_tknzed = bookinfo[cols_tknz].to_dict('series')
book_name, book_subname, category = book_tknzed['BName'], book_tknzed['BName_sub'],book_tknzed['Category']
tokens = np.array(list(itertools.chain(*book_name.values,*book_subname.values,*category.values)))
token_freq = np.unique(tokens,return_counts=True)

map_token_encode = make_encoding_by_freq(token_freq,size_feat=32000)
encode_1line =lambda x: list(map(lambda y : encode_tokens(map_token_encode,y),x))

In [23]:
#아래 ths 는 EDA 결과 제가 자의적으로 정한 내용
ths_author = int(np.round(len(book_name)/500)*75)
ths_publshr = int(np.round(len(book_name)/500)*5)

map_author_encode = make_author_encode_map(bookinfo[['Author','SalesPoint']],ths_author)
map_publshr_encode = make_publshr_encode_map(bookinfo['Publshr'],ths_publshr)

encode_maps = {
    'Author' : lambda x : encode_tokens(map_author_encode,x,oov=False),
    'Publshr' : lambda x : encode_tokens(map_publshr_encode,x,oov=False),
}

In [24]:
maxlens={
    'Category' : 5,
    'BName' : 30,
    'BName_sub' : 25
}
x_cols = ['BName', 'BName_sub', 'Author',
   'Author_mul', 'Publshr', 'Pdate', 'SalesPoint','Category']

book_cols = ['BName', 'BName_sub', 'Author', 'Author_mul', 'Publshr', 'Pdate',
       'RglPrice', 'SalesPoint', 'Category']
xcols_scalar = list(filter(lambda x : x not in cols_tknz,x_cols)) 

In [46]:
#encode X 
X_encoded=dict()
for mode,sample in tqdm(data_dict.items()):
    X_mode = sample['X']
    #padding and encoding
    encoded = pd.DataFrame(index=X_mode.index) 
    for col in cols_tknz :
        padded = pad_sequences(X_mode[col],padding='post',
                               maxlen=maxlens[col],
                               value='[PAD]',dtype=object)
        encoded[col] = list(np.apply_along_axis(encode_1line,0,padded))
    #concat encoded
    for col in tqdm(cols_freq):
        encoded[col] = X_mode[col].map(encode_maps[col])
    X_mode['Pdate'] = X_mode['Pdate'].astype(str)
#        date_format = '%Y-%m-%d'
    date_format = '%Y%m%d'
    encoded['Pdate']= pd.to_datetime(X_mode['Pdate'],format=date_format)
    encoded['Pdate']= pd_datetime_2_datenum(encoded['Pdate'])
    cols_else = list(filter(lambda x : x not in encoded.columns,x_cols))
    encoded[cols_else] = X_mode[cols_else]

    concat_tknzed =np.apply_along_axis(np.hstack,1,encoded[cols_tknz].values)
    x_scalar = encoded[xcols_scalar].values
    X = np.hstack((concat_tknzed,x_scalar))
    X_encoded[mode] = X    

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:06<00:00,  2.03s/it]


In [26]:
divmod(4034678,65)

(62071, 63)

In [27]:
cond = data_dict['trn']['y'] >= 200000
data_dict['trn']['X'][cond]

,BName,BName_sub,Author,Publshr,Author_mul,Pdate,SalesPoint,Category
58365,"[민, 음, 바칼로레아, 1, -, 50, 권, 세트]",[],로렝 드고,민음인,True,20060925,203,[청소년]
62802,"[식객, 세트, -, 전, 27, 권]","[(, 식객, 전, 27, 권, +, 식객, 매거진, )]",허영만,김영사,False,20100519,1844,[만화]
81677,"[슬램, 덩크, 완전, 판, 프리미엄, 세트, -, 전, 24, 권]","[(, 완결, ,, 묶음, )]",이노우에 다케히코,대원씨아이(만화),False,20080324,1145,[만화]
56529,"[로마인, 이야기, 세트, -, 전, 15, 권]",[],시오노 나나미,한길사,True,20070201,2088,[역사]
35000,"[이이화, 의, 한국사, 이야기, 1, ~, 22, 세트]",[],이이화,한길사,False,20040515,393,[역사]
157502,"[당신, 이, 보, 지, 못한, 희귀, 사진, -, 전, 3, 권]",[],쉬충마오,서해문집,False,20240705,2900,[역사]
157037,"[2024, 레저, 백서]",[],서천범,한국레저산업연구소,False,20240517,2115,"[건강, /, 취미]"
87253,"[한국, 현대사, 산책, 세트, -, 전, 23, 권]",[],강준만,인물과사상사,False,20110822,451,[역사]
59958,"[고우영, 수호지, 세트, -, 전, 20, 권]",[],고우영,자음과모음,False,20050101,414,[만화]
152740,"[토지, 1, ~, 20, 세트, -, 전, 20, 권]",[],박경리,다산책방,False,20230612,1854,"[소설, /, 시, /, 희곡]"


In [48]:
data_dict['trn']['X'].iloc[74310]

BName         [눈, 이, 아닌, 것, 으로, 도, 읽, 은, 기분]
BName_sub                                 []
Author                            요조 (Yozoh)
Publshr                                   난다
Author_mul                             False
Pdate                               20171230
SalesPoint                               666
Category                               [에세이]
Name: 114793, dtype: object

In [49]:
display(np.max(X_encoded['trn']))
display(np.min(X_encoded['trn']))
display(np.argmin(X_encoded['trn']))
X_encoded['trn'][62071]

17280000.0

-4603392.0

4034678

array([20, 1, 22, 0, 0, 18835, 31294, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True,
       271, -4603392.0, 101], dtype=object)

In [50]:
display(np.sum(np.isnan(X_encoded['trn'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['vld'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['tst'].astype(np.float64))))

0

0

0

In [30]:
len(np.unique(X_encoded['trn'][:,:60]))

31991

In [31]:
vocab = np.unique(X_encoded['tst'][:,:60])
cond = vocab[1:]-vocab[:-1] > 1

vocab[:-1][cond]

array([2592, 2794, 2883, ..., 31991, 31994, 31997], dtype=object)

In [32]:
max(vocab),min(vocab)

(32002, 0)

In [33]:
X_coded = {
    mode : data.astype(np.int32)
    for mode, data in X_encoded.items()
} 

In [34]:
import copy
scale_vec,scale_scalar = False, True
len_tknzed, len_scalar = sum(maxlens.values()), len(xcols_scalar)
scale_partition = [(0,len_tknzed)] + [(len_tknzed+i,len_tknzed+1+i) for i in range(len_scalar)]
scaler_list = [copy.deepcopy(MinMaxScaler())] if scale_vec else [None]
scaler_list += [copy.deepcopy(MinMaxScaler()) if scale_scalar else None for _ in scale_partition[1:]]
scalers = list(zip(scale_partition,scaler_list))


In [35]:
X_encoded['trn'].shape

(101173, 65)

In [55]:
scale_partition

[(0, 60), (60, 61), (61, 62), (62, 63), (63, 64), (64, 65)]

In [56]:
scalers

[((0, 60), None),
 ((60, 61), MinMaxScaler()),
 ((61, 62), MinMaxScaler()),
 ((62, 63), MinMaxScaler()),
 ((63, 64), MinMaxScaler()),
 ((64, 65), MinMaxScaler())]

In [57]:
X_scaled = dict()
for mode in ['trn','vld','tst']:
    intermid = []
    for (cols,scaler) in scalers:
        sliced = X_encoded[mode][:,cols[0]:cols[1]]
        sliced = sliced.astype(np.float64)
        if scaler is not None :
            if mode == 'trn': sliced = scaler.fit_transform(sliced)
            else : sliced = scaler.transform(sliced)
        intermid.append(sliced)

    X_scaled[mode] = np.hstack(intermid)

In [58]:
X_scaled['trn'].shape
X_scaled['trn'][0]

array([4.00000000e+00, 1.00000000e+00, 3.00000000e+00, 1.00000000e+00,
       5.00000000e+00, 3.36800000e+03, 1.70000000e+01, 7.58800000e+03,
       6.26000000e+02, 1.10000000e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [59]:
data_type = 'scaled-scalar'
strat=0
ver=1.0
dir_path = os.path.join(RSLT_DIR,'model_input')
for mode, x_scaled in X_scaled.items():
    save_pkl(dir_path,'{}.v{}_st-{}_X_{}.pkl'.format(data_type,ver,strat,mode),x_scaled)
for mode,sample in data_dict.items(): 
    save_pkl(dir_path,'{}.v{}_st-{}_y_{}.pkl'.format(data_type,ver,strat,mode),sample['y'].to_numpy())